# 决策树
决策树(decision tree)是一种基本的分类和回归方法.在分类问题中,决策树在对特征进行分类过程中可以被认为是一种 `if-then` 规则的集合.  
决策树优点:  
* 有良好的可读性
* 分类速度快  

决策树的学习包括:  
* 特征选择
* 决策树的生成
* 决策树的修剪  

决策树的算法:  
* ID3 : 信息增益
* C4.5 : 信息增益比
* CART : gini 指数

## 特征选择
### 1. 信息增益选择特征
选择合适的特征作为根节点能提升分类效果,特征的信息增益最大的为最优特征.  
信息增益 = 经验熵 - 条件经验熵  
$ g(D,A) = H(D) - H(D|A) $  
* 经验熵: H(D) 为数据集 D 的经验熵  
$ H(D) = -\sum_{i=1}^{n}p_i{log_2{p_i}} $  
$ p_i = P(X=x_i),i=1,2,...,n $  
* 经验条件熵: H(Y|X) 特征 X 对数据集 D 的经验熵  
$ H(Y|X) = \sum_{i=1}^{n}p_iH(Y|X=x_i) $  

示例:  
训练数据集 D,|D| 表示其样本容量,即样本个数.设 K 个类 $ C_k $,k=1,2,...,K,$ |C_k| $ 为属于类 $ C_k $ 的样本个数,$ \sum_{k=1}^{K}|C_k|=|D| $.社特征 A 有 n 个不同的取值 {$ a_1,a_2,...,a_n $},根据特征 A 的取值将 D 划分为 n 个子集 {$ D_1,D_2,...,D_3 $},$ |D_i| $ 为 $ D_i $ 的样本个数,$ \sum_{i=1}^{n}|D_i|=|D| $,子集 $ D_i $ 中数据类 $ C_k $ 的样本的集合为 $ D_{ik} $,即 $ D_{ik}={D_i}\bigcap{C_k},{D_{ik}} 为 D_{ik} $ 的样本个数.  
1. 计算数据集 D 的经验熵 H(D)  
$ H(D) = -\sum_{k=1}^{k}\frac{|C_k|}{|D|}log_2\frac{|C_k|}{|D|} $  
2. 计算特征 A 对数据集 D 的经验条件熵 H(D|A)  
$ H(D|A) = \sum_{i=1}^{n}\frac{|C_i|}{|D|}H(D_i) = -\sum_{i=1}^{n}\frac{|C_i|}{|D|}\sum_{k=1}^{k}\frac{|D_ik|}{|Di|}log_2\frac{|D_ik|}{|Di|} $  
3. 计算信息增益  
$ g(D,A) = H(D) - H(D|A) $  

### 2. 信息增益比选择特征  
信息增益比是对信息增益的校正.当分类困难,训练数据集的经验熵过大,信息增益值会偏大,反之偏小,这时就需要对其进行校正.  
* 信息增益比 = 信息增益 / 经验熵  
$ g_R(D,A) = \frac{g(D,A)}{H(D)} $  

## 决策树的生成  


In [2]:
import math
from math import log
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    columns = ['年龄', '有工作', '有自己的房子', '信贷情况', '类别']
    # 返回数据集和每个维度的名称
    return datasets, columns
datasets, columns = create_data()
train_data = pd.DataFrame(datasets, columns=columns)

In [9]:
def empiricalEntropy(train_data, label):
    """
    经验熵
    train_data: 数据集
    label: 目标集
    """
    data_length = len(train_data)
    labels_count = train_data[label].value_counts()
    pi = [label / data_length for label in labels_count]
    entropy = -sum([(p * log(p, 2)) for p in pi])
    return entropy

def empiricalConditionalEntropy(train_data, columns):
    """
    条件经验熵
    """
    data_length = len(train_data)
    labels = pd.unique(train_data[columns[-1]])
    labels_length = len(labels)
    result = {}
    for col in columns[:-1]:
        attr_name = pd.unique(train_data[col])
        attr_count = train_data[col].value_counts()
        tmp = []
        for name, count in zip(attr_name, attr_count):
            # 特征的一个属性属于某类的概率
            pi = [len(train_data[train_data[col] == name][train_data[columns[-1]] == label]) / count for label in labels]
            entropy = -sum([p * log(p, 2) for p in pi if p != 0.0]) * (count / data_length)
            tmp.append(entropy)
        result[col] = sum(tmp)
    max_entropy = sorted(result.items(), key=lambda x: x[1])[0]
    return max_entropy

def infoGain(emp_en, con_en):
    return emp_en - con_en

def main(train_data, label, columns):
    columns = columns + [label]
    emp_en = empiricalEntropy(train_data, label)
    con_en = empiricalConditionalEntropy(train_data, columns)
    result = infoGain(emp_en, con_en[1])
    text = '特征({})的信息增益最大为{}，选择为根节点特征'.format(con_en[0], result)
    return text

In [10]:
main(train_data, '类别', ['年龄', '有工作', '有自己的房子', '信贷情况'])

/home/dxigui/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


'特征(有自己的房子)的信息增益最大为0.4199730940219749，选择为根节点特征'